# 转换SDG网络数据为Gephi格式

本notebook将SDG现金流网络数据转换为Gephi可以直接导入的格式

## Gephi需要的格式：
1. **边表（Edges）**：Source, Target, Weight, Type
2. **节点表（Nodes）**：Id, Label（可选，包含节点属性）

## 生成两套网络数据：
1. **SDG子目标层面**（如1.1, 1.2, 8.1等）
2. **SDG大目标层面**（如1, 2, 8等，共17个）

In [19]:
import pandas as pd
import numpy as np

---
# Part 1: SDG子目标层面网络

## 1. 加载原始数据

In [20]:
# 读取SDG网络边数据
df = pd.read_csv('../../output/sdg_cashflow_network.csv', encoding='utf-8-sig')
print(f"原始数据形状: {df.shape}")
df.head()

原始数据形状: (9383, 7)


,source_sdg,target_sdg,negative_cashflow,positive_cashflow,negative_transaction_count,positive_transaction_count,net_cashflow
0,1.1,1.1,0.0,15029.0,0.0,23.0,15029.0
1,1.1,1.4,0.0,15021.0,0.0,21.0,15021.0
2,1.1,10.2,0.0,3337.0,0.0,3.0,3337.0
3,1.1,10.5,660.0,0.0,2.0,0.0,-660.0
4,1.1,11.1,0.0,3461.0,0.0,17.0,3461.0


## 2. 生成SDG子目标边表

In [21]:
# 创建边表（正负面影响分成两条边）
edges_list = []

for idx, row in df.iterrows():
    # 添加正面影响边（如果存在）
    if row['positive_cashflow'] > 0:
        edges_list.append({
            'Source': row['source_sdg'],
            'Target': row['target_sdg'],
            'Weight': row['positive_cashflow'],
            'Type': 'Directed',
            'Impact': 'Positive',
            'Transaction_Count': row['positive_transaction_count']
        })
    
    # 添加负面影响边（如果存在）
    if row['negative_cashflow'] > 0:
        edges_list.append({
            'Source': row['source_sdg'],
            'Target': row['target_sdg'],
            'Weight': row['negative_cashflow'],
            'Type': 'Directed',
            'Impact': 'Negative',
            'Transaction_Count': row['negative_transaction_count']
        })

gephi_edges_target = pd.DataFrame(edges_list)
print(f"\nSDG子目标边表形状: {gephi_edges_target.shape}")
print(f"正面影响边数: {(gephi_edges_target['Impact'] == 'Positive').sum()}")
print(f"负面影响边数: {(gephi_edges_target['Impact'] == 'Negative').sum()}")
gephi_edges_target.head(10)


SDG子目标边表形状: (10919, 6)
正面影响边数: 8339
负面影响边数: 2580


,Source,Target,Weight,Type,Impact,Transaction_Count
0,1.1,1.1,15029.0,Directed,Positive,23.0
1,1.1,1.4,15021.0,Directed,Positive,21.0
2,1.1,10.2,3337.0,Directed,Positive,3.0
3,1.1,10.5,660.0,Directed,Negative,2.0
4,1.1,11.1,3461.0,Directed,Positive,17.0
5,1.1,11.2,8468.0,Directed,Positive,12.0
6,1.1,11.3,9.0,Directed,Positive,1.0
7,1.1,11.4,6.0,Directed,Positive,2.0
8,1.1,11.6,280.0,Directed,Positive,3.0
9,1.1,11.7,34.0,Directed,Positive,2.0


## 3. 生成SDG子目标节点表

In [22]:
# 读取节点统计数据
nodes_df = pd.read_csv('../../output/sdg_summary_statistics.csv', encoding='utf-8-sig')
print(f"节点统计数据形状: {nodes_df.shape}")
nodes_df.head()

节点统计数据形状: (129, 6)


,sdg_target,positive_inflow,positive_outflow,negative_inflow,negative_outflow,net_flow
0,9.4,85991244.0,45636960.0,2487931.0,2099804.0,39966157.0
1,7.3,47862824.0,19238591.0,1161942.0,3292021.0,30754312.0
2,9.1,104181694.0,74559564.0,0.0,0.0,29622130.0
3,12.2,46104498.0,29776181.0,41262216.0,45306127.0,20372228.0
4,8.2,106289026.0,87428526.0,525640.0,500406.0,18835266.0


In [23]:
# 创建Gephi节点表
gephi_nodes_target = pd.DataFrame({
    'Id': nodes_df['sdg_target'],
    'Label': nodes_df['sdg_target'],
    'SDG_Goal': nodes_df['sdg_target'].apply(lambda x: str(x).split('.')[0]),  # 提取SDG大目标编号
    'Positive_Inflow': nodes_df['positive_inflow'],
    'Positive_Outflow': nodes_df['positive_outflow'],
    'Negative_Inflow': nodes_df['negative_inflow'],
    'Negative_Outflow': nodes_df['negative_outflow'],
    'Net_Flow': nodes_df['net_flow'],
    'Total_Flow': nodes_df['positive_inflow'] + nodes_df['positive_outflow'] + 
                  nodes_df['negative_inflow'] + nodes_df['negative_outflow']
})

# 添加节点分类（基于净流量）
gephi_nodes_target['Node_Type'] = gephi_nodes_target['Net_Flow'].apply(
    lambda x: 'Net_Receiver' if x > 0 else ('Net_Provider' if x < 0 else 'Balanced')
)

print(f"\nSDG子目标节点表形状: {gephi_nodes_target.shape}")
print(f"\n节点类型分布:")
print(gephi_nodes_target['Node_Type'].value_counts())
print(f"\nSDG大目标分布:")
print(gephi_nodes_target['SDG_Goal'].value_counts().sort_index())
print("\n节点表样例:")
gephi_nodes_target.head()


SDG子目标节点表形状: (129, 10)

节点类型分布:
Node_Type
Net_Provider    65
Net_Receiver    64
Name: count, dtype: int64

SDG大目标分布:
SDG_Goal
1      5
10     5
11    10
12    11
13     4
14     8
15     6
16    10
17    10
2      6
3     11
4      8
5      6
6      8
7      4
8     10
9      7
Name: count, dtype: int64

节点表样例:


,Id,Label,SDG_Goal,Positive_Inflow,Positive_Outflow,Negative_Inflow,Negative_Outflow,Net_Flow,Total_Flow,Node_Type
0,9.4,9.4,9,85991244.0,45636960.0,2487931.0,2099804.0,39966157.0,136215939.0,Net_Receiver
1,7.3,7.3,7,47862824.0,19238591.0,1161942.0,3292021.0,30754312.0,71555378.0,Net_Receiver
2,9.1,9.1,9,104181694.0,74559564.0,0.0,0.0,29622130.0,178741258.0,Net_Receiver
3,12.2,12.2,12,46104498.0,29776181.0,41262216.0,45306127.0,20372228.0,162449022.0,Net_Receiver
4,8.2,8.2,8,106289026.0,87428526.0,525640.0,500406.0,18835266.0,194743598.0,Net_Receiver


---
# Part 2: SDG大目标层面网络

## 4. 聚合到SDG大目标层面

In [24]:
# 提取SDG大目标编号
df_goal = df.copy()
df_goal['source_goal'] = df_goal['source_sdg'].apply(lambda x: str(x).split('.')[0])
df_goal['target_goal'] = df_goal['target_sdg'].apply(lambda x: str(x).split('.')[0])

# 按大目标聚合
df_goal_agg = df_goal.groupby(['source_goal', 'target_goal']).agg({
    'positive_cashflow': 'sum',
    'negative_cashflow': 'sum',
    'positive_transaction_count': 'sum',
    'negative_transaction_count': 'sum'
}).reset_index()

# 计算净现金流
df_goal_agg['net_cashflow'] = df_goal_agg['positive_cashflow'] - df_goal_agg['negative_cashflow']

print(f"\nSDG大目标聚合数据形状: {df_goal_agg.shape}")
print(f"涉及SDG大目标数: {df_goal_agg['source_goal'].nunique()}")
df_goal_agg.head()


SDG大目标聚合数据形状: (289, 7)
涉及SDG大目标数: 17


,source_goal,target_goal,positive_cashflow,negative_cashflow,positive_transaction_count,negative_transaction_count,net_cashflow
0,1,1,487154.0,48152.0,227.0,55.0,439002.0
1,1,10,1056019.0,62303.0,124.0,120.0,993716.0
2,1,11,1404826.0,120.0,999.0,5.0,1404706.0
3,1,12,402051.0,253823.0,1253.0,652.0,148228.0
4,1,13,254025.0,43548.0,284.0,112.0,210477.0


## 5. 生成SDG大目标边表

In [25]:
# 创建大目标边表（正负面影响分成两条边）
edges_goal_list = []

for idx, row in df_goal_agg.iterrows():
    # 添加正面影响边（如果存在）
    if row['positive_cashflow'] > 0:
        edges_goal_list.append({
            'Source': row['source_goal'],
            'Target': row['target_goal'],
            'Weight': row['positive_cashflow'],
            'Type': 'Directed',
            'Impact': 'Positive',
            'Transaction_Count': row['positive_transaction_count']
        })
    
    # 添加负面影响边（如果存在）
    if row['negative_cashflow'] > 0:
        edges_goal_list.append({
            'Source': row['source_goal'],
            'Target': row['target_goal'],
            'Weight': row['negative_cashflow'],
            'Type': 'Directed',
            'Impact': 'Negative',
            'Transaction_Count': row['negative_transaction_count']
        })

gephi_edges_goal = pd.DataFrame(edges_goal_list)
print(f"\nSDG大目标边表形状: {gephi_edges_goal.shape}")
print(f"正面影响边数: {(gephi_edges_goal['Impact'] == 'Positive').sum()}")
print(f"负面影响边数: {(gephi_edges_goal['Impact'] == 'Negative').sum()}")
gephi_edges_goal.head(10)


SDG大目标边表形状: (547, 6)
正面影响边数: 288
负面影响边数: 259


,Source,Target,Weight,Type,Impact,Transaction_Count
0,1,1,487154.0,Directed,Positive,227.0
1,1,1,48152.0,Directed,Negative,55.0
2,1,10,1056019.0,Directed,Positive,124.0
3,1,10,62303.0,Directed,Negative,120.0
4,1,11,1404826.0,Directed,Positive,999.0
5,1,11,120.0,Directed,Negative,5.0
6,1,12,402051.0,Directed,Positive,1253.0
7,1,12,253823.0,Directed,Negative,652.0
8,1,13,254025.0,Directed,Positive,284.0
9,1,13,43548.0,Directed,Negative,112.0


## 6. 生成SDG大目标节点表

In [26]:
# 从子目标层面聚合到大目标层面
goal_nodes_data = []

for goal in sorted(gephi_nodes_target['SDG_Goal'].unique(), key=lambda x: int(x)):
    goal_targets = gephi_nodes_target[gephi_nodes_target['SDG_Goal'] == goal]
    
    goal_nodes_data.append({
        'Id': goal,
        'Label': f'SDG {goal}',
        'Target_Count': len(goal_targets),  # 包含的子目标数量
        'Positive_Inflow': goal_targets['Positive_Inflow'].sum(),
        'Positive_Outflow': goal_targets['Positive_Outflow'].sum(),
        'Negative_Inflow': goal_targets['Negative_Inflow'].sum(),
        'Negative_Outflow': goal_targets['Negative_Outflow'].sum(),
        'Net_Flow': goal_targets['Net_Flow'].sum(),
        'Total_Flow': goal_targets['Total_Flow'].sum()
    })

gephi_nodes_goal = pd.DataFrame(goal_nodes_data)

# 添加节点分类
gephi_nodes_goal['Node_Type'] = gephi_nodes_goal['Net_Flow'].apply(
    lambda x: 'Net_Receiver' if x > 0 else ('Net_Provider' if x < 0 else 'Balanced')
)

print(f"\nSDG大目标节点表形状: {gephi_nodes_goal.shape}")
print(f"\n节点类型分布:")
print(gephi_nodes_goal['Node_Type'].value_counts())
print("\n节点表:")
gephi_nodes_goal


SDG大目标节点表形状: (17, 10)

节点类型分布:
Node_Type
Net_Provider    9
Net_Receiver    8
Name: count, dtype: int64

节点表:


,Id,Label,Target_Count,Positive_Inflow,Positive_Outflow,Negative_Inflow,Negative_Outflow,Net_Flow,Total_Flow,Node_Type
0,1,SDG 1,5,24517431.0,17888806.0,1579128.0,663794.0,5713291.0,44649159.0,Net_Receiver
1,2,SDG 2,6,49579554.0,66207422.0,2609870.0,2939417.0,-16298321.0,121336263.0,Net_Provider
2,3,SDG 3,11,32655397.0,71418896.0,36370310.0,38220452.0,-36913357.0,178665055.0,Net_Provider
3,4,SDG 4,8,13780211.0,29423909.0,134234.0,87152.0,-15690780.0,43425506.0,Net_Provider
4,5,SDG 5,6,1495174.0,1509790.0,588084.0,848746.0,246046.0,4441794.0,Net_Receiver
5,6,SDG 6,8,10964818.0,5518986.0,6895455.0,8012785.0,6563162.0,31392044.0,Net_Receiver
6,7,SDG 7,4,73818775.0,42376127.0,4811822.0,8780230.0,35411056.0,129786954.0,Net_Receiver
7,8,SDG 8,10,261973448.0,240837889.0,27017317.0,21459704.0,15577946.0,551288358.0,Net_Receiver
8,9,SDG 9,7,258862545.0,196986865.0,2720953.0,2154591.0,61309318.0,460724954.0,Net_Receiver
9,10,SDG 10,5,11387624.0,20125717.0,10511454.0,9635257.0,-9614290.0,51660052.0,Net_Provider


---
# Part 3: 保存所有文件

In [27]:
# 保存SDG子目标层面数据
gephi_edges_target.to_csv('../../output/gephi_edges_target.csv', index=False, encoding='utf-8-sig')
print("✓ 已保存: gephi_edges_target.csv (SDG子目标边表)")

gephi_nodes_target.to_csv('../../output/gephi_nodes_target.csv', index=False, encoding='utf-8-sig')
print("✓ 已保存: gephi_nodes_target.csv (SDG子目标节点表)")

# 保存SDG大目标层面数据
gephi_edges_goal.to_csv('../../output/gephi_edges_goal.csv', index=False, encoding='utf-8-sig')
print("✓ 已保存: gephi_edges_goal.csv (SDG大目标边表)")

gephi_nodes_goal.to_csv('../../output/gephi_nodes_goal.csv', index=False, encoding='utf-8-sig')
print("✓ 已保存: gephi_nodes_goal.csv (SDG大目标节点表)")

✓ 已保存: gephi_edges_target.csv (SDG子目标边表)
✓ 已保存: gephi_nodes_target.csv (SDG子目标节点表)
✓ 已保存: gephi_edges_goal.csv (SDG大目标边表)
✓ 已保存: gephi_nodes_goal.csv (SDG大目标节点表)


---
# Part 4: 数据统计与可视化指南

## 数据统计摘要

In [28]:
print("=" * 70)
print("Gephi数据转换完成！")
print("=" * 70)

print("\n【SDG子目标层面】")
print(f"  节点数: {len(gephi_nodes_target)}")
print(f"  边数: {len(gephi_edges_target)}")
print(f"  总权重: {gephi_edges_target['Weight'].sum():,.0f}")
print(f"  平均边权重: {gephi_edges_target['Weight'].mean():,.0f}")

print("\n【SDG大目标层面】")
print(f"  节点数: {len(gephi_nodes_goal)} (SDG 1-17)")
print(f"  边数: {len(gephi_edges_goal)}")
print(f"  总权重: {gephi_edges_goal['Weight'].sum():,.0f}")
print(f"  平均边权重: {gephi_edges_goal['Weight'].mean():,.0f}")

Gephi数据转换完成！

【SDG子目标层面】
  节点数: 129
  边数: 10919
  总权重: 1,315,732,994
  平均边权重: 120,499

【SDG大目标层面】
  节点数: 17 (SDG 1-17)
  边数: 547
  总权重: 1,315,732,994
  平均边权重: 2,405,362


In [29]:
print("\n=== Top 10 最大权重的边（子目标层面） ===")
print(gephi_edges_target.nlargest(10, 'Weight')[['Source', 'Target', 'Weight', 'Impact']])

print("\n=== Top 10 最大权重的边（大目标层面） ===")
print(gephi_edges_goal.nlargest(10, 'Weight')[['Source', 'Target', 'Weight', 'Impact']])


=== Top 10 最大权重的边（子目标层面） ===
      Source Target      Weight    Impact
2438    12.5   12.5  15630321.0  Negative
9467     8.2    8.2  10729800.0  Positive
10267    9.1    9.1   9938033.0  Positive
9838     8.5    8.5   9605599.0  Positive
9835     8.5    8.2   9508108.0  Positive
3296    13.2   12.5   9181775.0  Negative
1988    12.2   12.5   9020728.0  Negative
2432    12.5   12.2   8319480.0  Negative
10260    9.1    8.2   8278531.0  Positive
3309    13.2   13.2   8075524.0  Negative

=== Top 10 最大权重的边（大目标层面） ===
    Source Target      Weight    Impact
102     12     12  72831994.0  Negative
511      8      8  66807930.0  Positive
513      8      9  63202489.0  Positive
545      9      9  61397464.0  Positive
543      9      8  52730323.0  Positive
125     12      8  29117735.0  Positive
488      8     12  27612933.0  Positive
127     12      9  23727310.0  Positive
520      9     12  20004206.0  Positive
104     12     13  19461454.0  Negative


In [30]:
print("\n=== Top 10 最活跃的节点（子目标层面） ===")
print(gephi_nodes_target.nlargest(10, 'Total_Flow')[['Id', 'SDG_Goal', 'Total_Flow', 'Net_Flow']])

print("\n=== Top 10 最活跃的节点（大目标层面） ===")
print(gephi_nodes_goal.nlargest(10, 'Total_Flow')[['Id', 'Label', 'Total_Flow', 'Net_Flow']])


=== Top 10 最活跃的节点（子目标层面） ===
       Id SDG_Goal   Total_Flow    Net_Flow
4     8.2        8  194743598.0  18835266.0
128  12.5       12  183639182.0 -24289516.0
2     9.1        9  178741258.0  29622130.0
9     8.5        8  175857810.0   5688828.0
3    12.2       12  162449022.0  20372228.0
123  13.2       13  139055253.0 -11449001.0
0     9.4        9  136215939.0  39966157.0
108   8.3        8   83681807.0  -3629223.0
1     7.3        7   71555378.0  30754312.0
120   3.9        3   70165746.0 -10499600.0

=== Top 10 最活跃的节点（大目标层面） ===
    Id   Label   Total_Flow    Net_Flow
7    8   SDG 8  551288358.0  15577946.0
11  12  SDG 12  520677162.0 -14009556.0
8    9   SDG 9  460724954.0  61309318.0
2    3   SDG 3  178665055.0 -36913357.0
12  13  SDG 13  165808625.0 -21277895.0
6    7   SDG 7  129786954.0  35411056.0
1    2   SDG 2  121336263.0 -16298321.0
10  11  SDG 11  107763807.0 -11446669.0
14  15  SDG 15   98841685.0  17468417.0
15  16  SDG 16   57212649.0 -23642451.0


## Gephi导入指南

### 方案1：导入SDG子目标网络（129个节点）

1. **打开Gephi** → File → Import Spreadsheet

2. **导入边表**：
   - 选择 `gephi_edges_target.csv`
   - Import as: **Edges table**
   - Graph Type: **Directed**

3. **导入节点表**：
   - 选择 `gephi_nodes_target.csv`
   - Import as: **Nodes table**
   - Append to existing workspace

4. **可视化设置**：
   - **节点颜色**：按 `SDG_Goal` 着色（17种颜色代表17个SDG大类）
   - **节点大小**：按 `Total_Flow` 调整
   - **边颜色**：按 `Impact` 着色（蓝色=Positive，红色=Negative）
   - **边粗细**：按 `Weight` 调整

### 方案2：导入SDG大目标网络（17个节点）- 推荐用于概览

1. **新建Workspace**（避免与子目标网络混淆）

2. **导入边表**：
   - 选择 `gephi_edges_goal.csv`
   - Import as: **Edges table**
   - Graph Type: **Directed**

3. **导入节点表**：
   - 选择 `gephi_nodes_goal.csv`
   - Import as: **Nodes table**
   - Append to existing workspace

4. **可视化设置**：
   - **布局**：Circular Layout 或 Force Atlas 2
   - **节点颜色**：按 `Node_Type` 或使用自定义颜色
   - **节点大小**：按 `Total_Flow` 调整
   - **边颜色**：按 `Impact` 着色

### 高级技巧：

- **子目标网络中按SDG分组**：Filters → Attributes → Partition → SDG_Goal
- **社区检测**：Statistics → Modularity
- **中心性分析**：Statistics → Network Diameter（PageRank, Betweenness等）
- **导出高质量图片**：Preview → Export → SVG/PNG/PDF